In [61]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

# Load dataset
data = pd.read_csv('C:/Users/swaro/OneDrive/Documents/GitHub/Credit-Risk-Assessment-using-Feed-Forward-Neural-Networks/Datasets/mortgage.csv')

# Drop rows with missing values
data = data.dropna()

# Define features and target
features = [
    'balance_time', 'LTV_time', 'interest_rate_time', 'hpi_time',
    'gdp_time', 'uer_time', 'REtype_CO_orig_time', 'REtype_PU_orig_time',
    'REtype_SF_orig_time', 'investor_orig_time', 'FICO_orig_time',
    'LTV_orig_time', 'Interest_Rate_orig_time', 'hpi_orig_time'
]
target = 'default_time'

# Separate features (X) and target (y)
X = data[features].values
y = data[target].values

# Scale features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create sequences for LSTM input
max_seq_length = 5  # You can adjust this based on the dataset
sequences = []
targets = []

# Create sequences by sliding window
for i in range(len(data) - max_seq_length + 1):
    seq = X[i:i + max_seq_length]  # Get the sequence of features
    sequences.append(seq)
    targets.append(y[i + max_seq_length - 1])  # Use the last value of default_time as target

# Convert to numpy arrays
X = np.array(sequences)
y = np.array(targets)

# Reshape y to ensure it has the right dimensions
y = np.expand_dims(y, axis=-1)  # Now shape will be (num_samples, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check class distribution in the training set
unique, counts = np.unique(y_train, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Class Distribution in y_train:", class_distribution)

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.flatten())
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Build the LSTM model with added complexity and regularization
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_seq_length, len(features))))

# First LSTM layer with regularization
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(Dropout(0.3))

# Second LSTM layer with regularization
model.add(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(Dropout(0.3))

# Third LSTM layer, no return_sequences since it's the last LSTM layer
model.add(LSTM(32, kernel_regularizer=l2(0.001), recurrent_regularizer=l2(0.001)))
model.add(Dropout(0.3))

# Output layer for binary classification
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))

# Compile the model with a lower learning rate
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0005), metrics=['accuracy'])

# Fit the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=200,  # Increased epochs for potential longer training
    batch_size=32,  # Smaller batch size
    validation_split=0.1,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Make predictions
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Prepare predictions for evaluation
y_test_flat = y_test.flatten()

# Confusion Matrix and Classification Report
print("Confusion Matrix:")
print(confusion_matrix(y_test_flat, y_pred_binary))

print("Classification Report:")
print(classification_report(y_test_flat, y_pred_binary))


Class Distribution in y_train: {0: 485699, 1: 12073}


C:\Users\swaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\core\masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
14000/14000 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - accuracy: 0.5805 - loss: 0.6903 - val_accuracy: 0.4600 - val_loss: 0.7469
Epoch 2/200
14000/14000 ━━━━━━━━━━━━━━━━━━━━ 130s 9ms/step - accuracy: 0.5898 - loss: 0.6532 - val_accuracy: 0.6734 - val_loss: 0.5924
Epoch 3/200
14000/14000 ━━━━━━━━━━━━━━━━━━━━ 114s 8ms/step - accuracy: 0.6046 - loss: 0.6439 - val_accuracy: 0.5950 - val_loss: 0.6418
Epoch 4/200
14000/14000 ━━━━━━━━━━━━━━━━━━━━ 123s 9ms/step - accuracy: 0.5964 - loss: 0.6374 - val_accuracy: 0.6365 - val_loss: 0.6094
Epoch 5/200
 3815/14000 ━━━━━━━━━━━━━━━━━━━━ 1:29 9ms/step - accuracy: 0.5668 - loss: 0.6538

KeyboardInterrupt: 